In [1]:
import os

path = 'C:/Users/'+os.getlogin()+'/Google Drive/University/Dissertation'
datapath = 'E:/Dissertation Data'

os.chdir(path+'/Code')
os.getcwd()

'C:\\Users\\User\\Google Drive\\University\\Dissertation\\Code'

We collect lexical co-occurrence statistics on all words in
the English Wikipedia, using the WikiExtractor tool2 to retrieve
plain text from the April 2015 dump (ca. 2.8B words),
and using simple regular expressions to segment sentences
and words, and remove URLs and punctuation. We perform
no POS tagging, lemmatisation, case normalisation,
or removal of numbers or symbols.

In [2]:
import pandas as pd
import re
import numpy as np

import nltk
from nltk.tokenize import MWETokenizer

from glove import Corpus, Glove

from gensim.models import Word2Vec

In [3]:
# On Simple English wiki

sf = open(datapath+'/Corpora/wiki/simple_20200601/simple_20200601_v2.txt', 'r', encoding='utf-8')

for lines in range(5):
    print(sf.readline())

April is the fourth month of the year, and comes between March and May. 

 It is one of four months to have 30 days. 

 April always begins on the same day of week as July, and additionally, January in leap years. 

 April always ends on the same day of the week as December. 

 April's flowers are the Sweet Pea and Daisy. 



In [4]:
from nltk.corpus.reader.plaintext import PlaintextCorpusReader

from nltk.tokenize import WhitespaceTokenizer

In [5]:
simp = PlaintextCorpusReader(datapath+'/Corpora/wiki/simple_20200601/','simple_20200601_v2.txt',
                            word_tokenizer = WhitespaceTokenizer()
                            )

In [6]:
# Import word and sentence generators

from generators import sent_gen, word_gen, Sent_Seq

We collect word frequency information with the
SRILM language modelling toolkit (Stolcke, 2002), counting
n-grams (n <= 3), treating MWEs as contiguous bigrams
and trigrams), and identify MWE candidates by computing
the Poisson collocation measure (Quasthoff and Wolff,
2002) for all bigrams and trigrams (ca. 23M n-grams).
This method should be readily extensible to include longer
n-grams.

In [7]:
# Collate n-grams

from nltk.collocations import BigramCollocationFinder, TrigramCollocationFinder

from nltk.metrics import (
    BigramAssocMeasures,
    TrigramAssocMeasures,
    NgramAssocMeasures,
)

from nltk.metrics.spearman import (
    spearman_correlation,
    ranks_from_scores,
)

In [8]:
# Load n-gram dataframe produced in word2vec iteration

ngram_df = pd.read_pickle(datapath+'/Corpora/wiki/simple_20200601/ngram_df.pkl')

In [9]:
ngram_df

,ngram,freq,poisson,len
0,"($-125,050, $17,122,936)",1,-2.595935e+01,2
1,"(voleyu, spayana)",1,-2.595935e+01,2
2,"(vote3, Jeff5/10)",1,-2.595935e+01,2
3,"(vote11, Colby3/5)",1,-2.595935e+01,2
4,"(vorskane, Lalvar)",1,-2.595935e+01,2
...,...,...,...,...
23342685,"(and, the)",50287,-2.520684e+06,2
23342686,"(to, the)",62517,-3.073387e+06,2
23342687,"(is, a)",83617,-3.944892e+06,2
23342688,"(in, the)",157322,-7.610418e+06,2


We then automatically score the million most strongly associated
n-grams (i.e., roughly the top 5% of the Poisson-ranked
list) for compositionality.

Using word2vec (Mikolov et al., 2013) with the parameters
found to be most effective by Baroni et al. (2014), we
build a word embedding vector for every simplex word in
the vocabulary (ca. 1M types), as well as for each MWE candidate.

* Continuous bag of words model with 400-dimensional vectors, window size 5, subsampling with t = 10^-5, negative sampling with 10 samples. We build vectors only for tokens observed 20 times or more in the corpus.

We then compute the cosine similarity of the vector
representation for a MWE candidate with the vectors of its
constituent words, and take the arithmetic mean. 
In scoring
the compositionality of a candidate, we do not measure the
cosine similarity of the MWE with any stop words it may
contain, as stop words may be assumed to be semantically
uninformative.
* Stop words are taken here to be the 50 most frequent words in the vocabulary.

In [10]:
# Stopwords from corpus - 50 most frequent
from nltk import FreqDist

fdist = FreqDist(word_gen(simp, sent_mark=''))

stop = set( word for word, f in fdist.most_common(50))

In [11]:
stop

{'0',
 '1',
 '2',
 '3',
 '4',
 'A',
 'American',
 'He',
 'In',
 'It',
 'References',
 'The',
 'This',
 'a',
 'also',
 'an',
 'and',
 'are',
 'as',
 'at',
 'be',
 'born',
 'by',
 'can',
 'first',
 'for',
 'from',
 'had',
 'has',
 'have',
 'he',
 'his',
 'in',
 'is',
 'it',
 'not',
 'of',
 'on',
 'one',
 'or',
 'people',
 'that',
 'the',
 'their',
 'they',
 'to',
 'was',
 'were',
 'which',
 'with'}

In [12]:
from batcher import batcher  # Custom module with logic for assigning n-grams to batches, avoiding overlap

In [13]:
min_freq = 20

In [14]:
# Duplicate entries appearing for some reason. Removing here
ngram_df2 = ngram_df[ngram_df.freq >= min_freq].drop_duplicates().reset_index(drop=True)

ngram_df2

,ngram,freq,poisson,len
0,"(Ving, Rhames)",20,-6.056256e+02,2
1,"(Grădina, Zoologică)",20,-6.056256e+02,2
2,"(Gharb-Chrarda-Beni, Hssen)",20,-6.056256e+02,2
3,"(Karlovy, Vary)",20,-6.070334e+02,2
4,"(waystations, shuku-eki)",20,-6.070334e+02,2
...,...,...,...,...
215905,"(and, the)",50287,-2.520684e+06,2
215906,"(to, the)",62517,-3.073387e+06,2
215907,"(is, a)",83617,-3.944892e+06,2
215908,"(in, the)",157322,-7.610418e+06,2


In [15]:
eval_count = 150000

ngram_eval = ngram_df2[0:eval_count]

ngram_eval

,ngram,freq,poisson,len
0,"(Ving, Rhames)",20,-605.625590,2
1,"(Grădina, Zoologică)",20,-605.625590,2
2,"(Gharb-Chrarda-Beni, Hssen)",20,-605.625590,2
3,"(Karlovy, Vary)",20,-607.033377,2
4,"(waystations, shuku-eki)",20,-607.033377,2
...,...,...,...,...
149995,"(first, country, to)",45,-3071.582792,3
149996,"(from, A)",58,-3071.612088,2
149997,"(1998, 8)",64,-3071.704678,2
149998,"(had, tested, positive)",55,-3071.728206,3


In [16]:
# Clean up
del ngram_df, ngram_df2

In [17]:
batches, batch_count = batcher(ngram_eval.ngram, stopwords=stop, max_batches = 15)

batches

{('Ving', 'Rhames'): 1,
 ('Grădina', 'Zoologică'): 1,
 ('Gharb-Chrarda-Beni', 'Hssen'): 1,
 ('Karlovy', 'Vary'): 1,
 ('waystations', 'shuku-eki'): 1,
 ('Edinson', 'Cavani'): 1,
 ('Jule', 'Styne'): 1,
 ('Flèche', 'Wallonne'): 1,
 ('maillot', 'jaune'): 1,
 ('Puffy', 'AmiYumi'): 1,
 ('Aspleniaceae', 'Asplenium'): 1,
 ('Sheela', 'Shrinivas'): 1,
 ('Caid', 'Essebsi'): 1,
 ('Bellas', 'Artes'): 1,
 ('Sonu', 'Nigam'): 1,
 ('Päijänne', 'Tavastia'): 1,
 ('Lemony', 'Snicket'): 1,
 ('Trisha', 'Yearwood'): 1,
 ('Pim', 'Fortuyn'): 1,
 ('Dany', 'Heatley'): 1,
 ('BNP', 'Paribas'): 1,
 ('Jomo', 'Kenyatta'): 1,
 ('Liev', 'Schreiber'): 1,
 ('Bertolt', 'Brecht'): 1,
 ('bel', 'canto'): 1,
 ('Dukla', 'Jihlava'): 1,
 ('Lieke', 'Martens'): 1,
 ('Primorje-Gorski', 'Kotar'): 1,
 ('Luchino', 'Visconti'): 1,
 ('mutually', 'intelligible'): 1,
 ('conventionally', 'delimit'): 1,
 ("Giant's", 'Causeway'): 1,
 ('Ruhollah', 'Khomeini'): 1,
 ('Jair', 'Bolsonaro'): 1,
 ('Tomáš', 'Berdych'): 1,
 ('Hawaiian–Emperor', 'seam

In [18]:
len(batches)

150000

In [19]:
# Should be able to add batch information using df.map() but am encountering errors apparently relating
#  to indexing - workaround (though slower).

ngb_cols = ["ngram", "batch"]
rows = []

for ng in ngram_eval['ngram']:
    rows.append({"ngram" : ng,
                "batch" : batches[ng]})
    
ng_batch = pd.DataFrame(rows, columns = ngb_cols)

ng_batch

,ngram,batch
0,"(Ving, Rhames)",1
1,"(Grădina, Zoologică)",1
2,"(Gharb-Chrarda-Beni, Hssen)",1
3,"(Karlovy, Vary)",1
4,"(waystations, shuku-eki)",1
...,...,...
149995,"(first, country, to)",2
149996,"(from, A)",-2
149997,"(1998, 8)",6
149998,"(had, tested, positive)",4


In [20]:
ngram_eval = ngram_eval.merge(ng_batch, on='ngram')

ngram_eval

,ngram,freq,poisson,len,batch
0,"(Ving, Rhames)",20,-605.625590,2,1
1,"(Grădina, Zoologică)",20,-605.625590,2,1
2,"(Gharb-Chrarda-Beni, Hssen)",20,-605.625590,2,1
3,"(Karlovy, Vary)",20,-607.033377,2,1
4,"(waystations, shuku-eki)",20,-607.033377,2,1
...,...,...,...,...,...
149995,"(first, country, to)",45,-3071.582792,3,2
149996,"(from, A)",58,-3071.612088,2,-2
149997,"(1998, 8)",64,-3071.704678,2,6
149998,"(had, tested, positive)",55,-3071.728206,3,4


In [21]:
del ng_batch, batches

In [28]:
# Pickle evaluation frame (with batch numbers!) for later
ngram_eval.to_pickle(datapath+'/Corpora/wiki/simple_20200601/ngram_eval.pkl')

In [10]:
# If skipping creating ngram_eval

#ngram_eval = pd.read_pickle(datapath+'/Corpora/wiki/simple_20200601/ngram_eval.pkl')

In [11]:
ngram_eval.batch.value_counts()

 1     45301
 2     33312
 3     24686
 4     16668
 5     10517
 6      7502
 7      4240
 8      2702
 9      1905
 10     1051
-2       832
 11      450
-1       300
 12      247
 13      123
 14      107
 15       57
Name: batch, dtype: int64

In [12]:
from nltk.tokenize import MWETokenizer
from scipy.spatial.distance import cosine
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Flatten down to a single number
def cosim(x,y):
    return cosine_similarity(x.reshape(1,-1), y.reshape(1,-1))[0][0]

In [13]:
def mwe_score(exp, model, stats_frame):
    # Combined token for MWE
    mwetoken = '+'.join(exp)

    # Stopwords - 1 if component is a stopword, 0 if present, -1 if simplex word missing from vocab, -2 if MWE missing
    sws = []
    # Component vectors
    cvs = []

    #  Neighbours in original & MWE-aware space
    oldn = []
    newn = []

    # List of individual word similarities (where present in the vocab)
    css = []

    # Empty array
    earr = np.empty(1000)
    earr[:] = np.nan

    # Check that combined token exists in the vocab. This protects against inflation of n-gram counts caused by repeats
    #  of the same token (e.g. in lists like https://simple.wikipedia.org/wiki/List_of_cities,_towns_and_villages_in_Fars_Province)
    if mwetoken in model.dictionary:

        mwv = model.word_vectors[model.dictionary[mwetoken]]

        for w in exp:
            if w in model.dictionary:
                cvs.append(model.word_vectors[model.dictionary[w]])

                oldn.append(model.most_similar(w, number=5))

                if w in stop:
                    sws.append(1)
                    css.append(np.nan)
                else:
                    sws.append(0)
                    css.append(cosim(model.word_vectors[model.dictionary[w]], mwv ))

            # If component is absent from vocab
            else:
                sws.append(-1)
                cvs.append(earr)
                css.append(np.nan)

                oldn.append([])

        #  Mean cosim
        if min(sws) >= 0:
            cs = np.nanmean(css)
        else:
            cs = np.nan

        newn = model.most_similar(mwetoken, number=5)

    # Combined token missing from vocab - mark with defaults
    else:
        sws = [-2]
        mwv = np.empty(400)
        mwv[:] = np.nan


    # Append to stats df
    return stats_frame.append({
        'ngram'  : exp,
        'stopwords' : sws,
        'mwe_vector' : mwv,
        'component_vectors' : cvs,
        'component_cosims'  : css,
        'cosine_sim'  : cs,
        'base_nearest': oldn,
        'mwe_nearest' : newn,
    }, ignore_index=True)

In [14]:
#batch_count = 1

In [15]:
%%time

batch_dfs = {}

for bb in range(batch_count):
    print('Processing batch {} of {}'.format(bb+1,batch_count))
    
    # Subset DataFrame
    batch_dfs[bb] = ngram_eval[ngram_eval.batch == bb+1].reset_index(drop=True)
    
    # Initialise MWETokenizer
    batch_token_mwe = MWETokenizer(list(batch_dfs[bb].ngram) , separator='+')
    
    # Build model
    print('Building GloVe model')
    simp_corp = Corpus()

    sents_mwe = Sent_Seq(simp, batch_token_mwe)
    simp_corp.fit( sents_mwe , window = 10)
    
    batch_model = Glove(no_components = 300, 
             learning_rate = 0.05)
    
    print(' Training model')
    batch_model.fit(simp_corp.matrix, 
          epochs=25,
          no_threads=8,
          verbose=False)

    batch_model.add_dictionary(simp_corp.dictionary)

    # Save model
    print(' Saving model')
    batch_model.save(datapath+'/Models/2 GloVe/simple_batch{}_v2.model'.format(bb+1))
    # Reload looks like    new_model = Glove.load('glove.model')
    
    # Just saving models for now
    
batch_dfs

Processing batch 1 of 1
Building GloVe model
 Training model
 Saving model
Wall time: 38min 8s


{0:                              ngram  freq      poisson  len  batch
 0                   (Ving, Rhames)    20  -605.625590    2      1
 1             (Grădina, Zoologică)    20  -605.625590    2      1
 2      (Gharb-Chrarda-Beni, Hssen)    20  -605.625590    2      1
 3                  (Karlovy, Vary)    20  -607.033377    2      1
 4         (waystations, shuku-eki)    20  -607.033377    2      1
 ...                            ...   ...          ...  ...    ...
 45296             (the, South, of)    41 -3070.727239    3      1
 45297                   (no, such)    64 -3070.763377    2      1
 45298      (very, important, part)    49 -3070.800963    3      1
 45299        (a, children's, book)    49 -3071.464079    3      1
 45300              (to, get, them)    46 -3071.464786    3      1
 
 [45301 rows x 5 columns]}

In [16]:
# Model inspection
batch_model.most_similar('relationships', number=6)

[('relationship', 0.7570696300178639),
 ('Doctor-patient', 0.7032971906384414),
 ('rivalry', 0.6864205478742692),
 ('connections', 0.6742437997045764),
 ('ties', 0.6734428749018191)]

In [17]:
batch_model.most_similar('banana', number=6)

[('cacao', 0.7070576211561542),
 ('neuroectodermal', 0.6924046077134078),
 ('Condy’s', 0.6536899321462041),
 ('coconut', 0.643141442258816),
 ('tubers', 0.6293274227757036)]

In [18]:
simp_corp.matrix

<1059614x1059614 sparse matrix of type '<class 'numpy.float64'>'
	with 60555488 stored elements in COOrdinate format>

In [19]:
batch_model.dictionary['relationships']

18307

In [22]:
simp_corp.matrix.asformat('csr')[18307]

<1x1059614 sparse matrix of type '<class 'numpy.float64'>'
	with 1121 stored elements in Compressed Sparse Row format>

In [23]:
batch_model.dictionary['American']

307

In [24]:
batch_model.dictionary['man']

2130

In [29]:
simp_corp.matrix.asformat('csr')[2130,307]

0.0

In [ ]:
# Removed
    #print('Gathering MWE stats')
    # For each MWE, evaluate stats. Record vectors (in case we want to calculate different metrics later).
    #statsf = pd.DataFrame(columns=['ngram', 'stopwords', 'mwe_vector', 'component_vectors', 'component_cosims', 
    #                               'cosine_sim', 'base_nearest', 'mwe_nearest'])

    #for exp in batch_dfs[bb].ngram:
    #    statsf = mwe_score(exp,batch_model,statsf)

    #  Join back onto DataFrame
    #batch_dfs[bb] = batch_dfs[bb].merge(statsf, on='ngram')

In [ ]:
# Merge dataframes, sort by compositionality metric, export

# Also want the default batches with batch no < 0
all_batches = ngram_eval[ngram_eval.batch < 0]

for d in range(batch_count):
    all_batches = all_batches.append(batch_dfs[d])
    
all_batches = all_batches.sort_values('cosine_sim')
all_batches = all_batches.reset_index(drop=True)

all_batches.to_csv(datapath+'/Models/2 GloVe/Results/simple_output_001.csv', index=False)

In [ ]:
all_batches